# Nestkast Soort-Herkenning Model Training

Train een MobileNetV2 model dat:
1. Werkt op dag EN nacht screenshots
2. Soorten herkent (niet alleen bezet/leeg)

**Klassen:**
- `leeg` - geen vogel aanwezig
- `koolmees` - Parus major
- (later uit te breiden met pimpelmees, etc.)

**Stappen:**
1. Upload screenshots naar Google Drive
2. Organiseer in mappen per klasse
3. Train model
4. Download en deploy

In [ ]:
# Stap 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Stap 2: Maak mappenstructuur voor training data
import os

BASE_DIR = '/content/drive/MyDrive/EMSN/nestbox_species'
CLASSES = ['leeg', 'koolmees']  # Later uit te breiden

# Maak mappen
for split in ['train', 'val']:
    for cls in CLASSES:
        path = os.path.join(BASE_DIR, split, cls)
        os.makedirs(path, exist_ok=True)
        print(f'Created: {path}')

print('\n=== INSTRUCTIES ===')
print('Upload je screenshots naar deze mappen:')
print(f'  {BASE_DIR}/train/leeg/     <- lege nestkast screenshots (dag+nacht)')
print(f'  {BASE_DIR}/train/koolmees/ <- koolmees screenshots (dag+nacht)')
print(f'  {BASE_DIR}/val/leeg/       <- validatie lege (10-20%)')
print(f'  {BASE_DIR}/val/koolmees/   <- validatie koolmees (10-20%)')
print('\nTip: Gebruik zowel dag (kleur) als nacht (IR/zwart-wit) screenshots!')

In [ ]:
# Stap 3: Check hoeveel afbeeldingen we hebben
from pathlib import Path

print('=== Training Data Overzicht ===')
total = 0
for split in ['train', 'val']:
    print(f'\n{split.upper()}:')
    for cls in CLASSES:
        path = Path(BASE_DIR) / split / cls
        count = len(list(path.glob('*.jpg'))) + len(list(path.glob('*.png')))
        total += count
        status = '✓' if count >= 10 else '⚠️ te weinig'
        print(f'  {cls}: {count} afbeeldingen {status}')

print(f'\nTotaal: {total} afbeeldingen')
if total < 40:
    print('\n⚠️ Minimaal 40 afbeeldingen aanbevolen (20+ per klasse)')

In [ ]:
# Stap 4: Imports en configuratie
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import time

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

# Configuratie
INPUT_SIZE = 224
BATCH_SIZE = 16
NUM_EPOCHS = 30
LEARNING_RATE = 0.001

In [ ]:
# Stap 5: Data transformaties (met augmentatie voor robuustheid)

# Training: met augmentatie
train_transforms = transforms.Compose([
    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),  # Helpt voor dag/nacht variatie
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Validatie: geen augmentatie
val_transforms = transforms.Compose([
    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Laad datasets
train_dataset = datasets.ImageFolder(os.path.join(BASE_DIR, 'train'), transform=train_transforms)
val_dataset = datasets.ImageFolder(os.path.join(BASE_DIR, 'val'), transform=val_transforms)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Klassen
class_names = train_dataset.classes
num_classes = len(class_names)
print(f'Klassen: {class_names}')
print(f'Training samples: {len(train_dataset)}')
print(f'Validation samples: {len(val_dataset)}')

In [ ]:
# Stap 6: Bekijk een paar voorbeelden
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title:
        plt.title(title)

# Pak een batch
images, labels = next(iter(train_loader))

# Toon grid
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
for idx, ax in enumerate(axes.flat):
    if idx < len(images):
        img = images[idx].numpy().transpose((1, 2, 0))
        img = np.array([0.229, 0.224, 0.225]) * img + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)
        ax.imshow(img)
        ax.set_title(class_names[labels[idx]])
        ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Stap 7: Maak het model (MobileNetV2 met transfer learning)

def create_model(num_classes):
    # Laad pretrained MobileNetV2
    model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
    
    # Freeze eerste lagen (behoud ImageNet features)
    for param in model.features[:14].parameters():
        param.requires_grad = False
    
    # Vervang classifier
    num_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2),
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(128, num_classes)
    )
    
    return model

model = create_model(num_classes)
model = model.to(device)

# Loss en optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

print(f'Model klaar voor training met {num_classes} klassen: {class_names}')

In [ ]:
# Stap 8: Training functie

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    best_acc = 0.0
    best_model_state = None
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    for epoch in range(num_epochs):
        start_time = time.time()
        
        # Training fase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
        
        # Validatie fase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * images.size(0)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
        
        scheduler.step()
        
        # Bereken metrics
        train_loss = train_loss / train_total
        train_acc = 100. * train_correct / train_total
        val_loss = val_loss / val_total
        val_acc = 100. * val_correct / val_total
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        elapsed = time.time() - start_time
        
        # Bewaar beste model
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_state = model.state_dict().copy()
            marker = ' ⭐ BEST'
        else:
            marker = ''
        
        print(f'Epoch {epoch+1}/{num_epochs} ({elapsed:.1f}s) - '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.1f}% | '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.1f}%{marker}')
    
    # Laad beste model
    model.load_state_dict(best_model_state)
    return model, history, best_acc

print('Training functie klaar. Start training in volgende cel.')

In [ ]:
# Stap 9: Train het model!
print('=== START TRAINING ===')
print(f'Epochs: {NUM_EPOCHS}, Batch size: {BATCH_SIZE}, Learning rate: {LEARNING_RATE}')
print()

model, history, best_acc = train_model(
    model, train_loader, val_loader, 
    criterion, optimizer, scheduler, 
    NUM_EPOCHS
)

print(f'\n=== TRAINING VOLTOOID ===')
print(f'Beste validatie accuracy: {best_acc:.1f}%')

In [ ]:
# Stap 10: Plot training geschiedenis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Loss
ax1.plot(history['train_loss'], label='Train')
ax1.plot(history['val_loss'], label='Validation')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training & Validation Loss')
ax1.legend()
ax1.grid(True)

# Accuracy
ax2.plot(history['train_acc'], label='Train')
ax2.plot(history['val_acc'], label='Validation')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training & Validation Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Stap 11: Test op een paar afbeeldingen
model.eval()

# Pak validatie afbeeldingen
images, labels = next(iter(val_loader))
images, labels = images.to(device), labels.to(device)

with torch.no_grad():
    outputs = model(images)
    probs = torch.softmax(outputs, dim=1)
    _, preds = outputs.max(1)

# Toon resultaten
fig, axes = plt.subplots(2, 4, figsize=(14, 7))
for idx, ax in enumerate(axes.flat):
    if idx < len(images):
        img = images[idx].cpu().numpy().transpose((1, 2, 0))
        img = np.array([0.229, 0.224, 0.225]) * img + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)
        ax.imshow(img)
        
        pred_class = class_names[preds[idx]]
        true_class = class_names[labels[idx]]
        confidence = probs[idx][preds[idx]].item() * 100
        
        color = 'green' if pred_class == true_class else 'red'
        ax.set_title(f'Pred: {pred_class} ({confidence:.0f}%)\nTrue: {true_class}', color=color)
        ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Stap 12: Sla het model op
import datetime

# Model metadata
checkpoint = {
    'model_state_dict': model.state_dict(),
    'classes': class_names,
    'num_classes': num_classes,
    'architecture': 'mobilenet_v2',
    'input_size': INPUT_SIZE,
    'best_val_acc': best_acc,
    'train_samples': len(train_dataset),
    'val_samples': len(val_dataset),
    'epochs': NUM_EPOCHS,
    'trained_at': datetime.datetime.now().isoformat(),
    'model_type': 'species_recognition',  # Nieuw: geeft aan dat dit soort-herkenning is
    'supports_day_night': True  # Nieuw: werkt op dag en nacht
}

# Sla op in Drive
model_path = os.path.join(BASE_DIR, 'nestbox_species_model.pt')
torch.save(checkpoint, model_path)
print(f'Model opgeslagen: {model_path}')
print(f'Klassen: {class_names}')
print(f'Beste accuracy: {best_acc:.1f}%')
print()
print('=== VOLGENDE STAPPEN ===')
print('1. Download het model van Google Drive')
print('2. Upload naar: /mnt/nas-birdnet-archive/nestbox/models/')
print('3. Update de detector om dit model te gebruiken')

In [ ]:
# Stap 13: Test functie voor nieuwe afbeeldingen
def predict_image(image_path, model, class_names):
    """Test een enkele afbeelding"""
    transform = transforms.Compose([
        transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        probs = torch.softmax(outputs, dim=1)
        confidence, predicted = probs.max(1)
    
    pred_class = class_names[predicted.item()]
    conf = confidence.item() * 100
    
    # Toon afbeelding en resultaat
    plt.figure(figsize=(6, 6))
    plt.imshow(Image.open(image_path))
    plt.title(f'{pred_class} ({conf:.1f}%)', fontsize=14)
    plt.axis('off')
    plt.show()
    
    # Toon alle probabiliteiten
    print('Probabiliteiten:')
    for i, cls in enumerate(class_names):
        print(f'  {cls}: {probs[0][i].item()*100:.1f}%')
    
    return pred_class, conf

# Voorbeeld gebruik:
# predict_image('/content/drive/MyDrive/test_image.jpg', model, class_names)

## Klaar!

Het model is getraind en opgeslagen. Volgende stappen:

1. **Download** `nestbox_species_model.pt` van Google Drive
2. **Upload** naar de Pi/NAS
3. **Update** de detector om soorten te herkennen

### Later uitbreiden

Om nieuwe soorten toe te voegen:
1. Maak nieuwe map in train/ en val/ (bijv. `pimpelmees`)
2. Voeg screenshots toe
3. Hertraining het model

### Tips
- Gebruik minimaal 20 afbeeldingen per klasse
- Mix dag en nacht screenshots
- Voeg variatie toe (verschillende hoeken, lichtomstandigheden)